In [ ]:
# importa as bibliotecas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import plotly

In [ ]:
# mostra a data e o horário inicial do código
inicio_codigo = time.strftime('"%A, %d %B %Y, %H:%M:%S', time.localtime())

tempo_inicial = int(time.time())

print(inicio_codigo)

In [ ]:
# abre o navegador
nav = webdriver.Chrome()

r"""Caso não funcione: 
Baixe a nova versão do chromedriver no site:
https://chromedriver.chromium.org/downloads
Coloque o arquivo na pasta onde está instalado o anaconda. 
Por exemplo:
C:\Users\cesar\anaconda3
Rode o código de novo.
"""

In [ ]:
# abre o site e espera ele carregar
nav.implicitly_wait(10)
nav.get('http://loterias.caixa.gov.br/wps/portal/loterias/landing/megasena')

In [ ]:
# pega o número e a data do último concurso
def funcao_pegar_concurso_data():
    global concurso_data
    concurso_data = nav.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span').text
    return concurso_data

funcao_pegar_concurso_data()

In [ ]:
# faz a limpeza dos dados extraindo somente o número do último concurso
def funcao_pegar_concurso():
    global concurso
    concurso = int(concurso_data[concurso_data.find(' ')+1:concurso_data.rfind(' ')])
    return concurso

funcao_pegar_concurso()

In [ ]:
# faz a limpeza dos dados extraindo somente a data do último concurso
def funcao_pegar_data():
    global data
    data = concurso_data[concurso_data.find('(')+1:concurso_data.rfind(')')]
    return data

funcao_pegar_data()

In [ ]:
# pega quais foram os números sorteados no último concurso
def funcao_pegar_numeros_sorteio():
    global numeros_sorteio
    numeros_sorteio = nav.find_element_by_xpath('//*[@id="ulDezenas"]').text
    return numeros_sorteio

funcao_pegar_numeros_sorteio()

In [ ]:
# pega a quantidade de ganhadores e o valor do prêmio
def funcao_pegar_ganhador_premio():
    global ganhador_premio
    ganhador_premio = ""
    for item in range(1, 4):
        if ganhador_premio == "":
            ganhador_premio = nav.find_element_by_xpath(f'//*[@id="conteudoresultado"]/div[3]/div/p[1]/span[{item}]').text
        else:
            pass
    return ganhador_premio

funcao_pegar_ganhador_premio()

In [ ]:
# faz a limpeza dos dados extraindo somente a quantidade de ganhadores
def funcao_pegar_quantidade_ganhadores():
    global quantidade_ganhadores
    if len(ganhador_premio) > 20:
        quantidade_ganhadores = int(ganhador_premio[:ganhador_premio.find(' ')])
    else:
        quantidade_ganhadores = 0
    return quantidade_ganhadores

funcao_pegar_quantidade_ganhadores()

In [ ]:
# faz a limpeza dos dados extraindo somente o valor do prêmio
def funcao_pegar_premio():
    global premio
    if len(ganhador_premio) > 20:
        premio = ganhador_premio[ganhador_premio.rfind(' ')+1:]
    else:
        premio = '0'
    return premio

funcao_pegar_premio()

In [ ]:
# cria o DataFrame
df = pd.DataFrame([],
                  index=range(1, concurso+1),
                  columns = ['concurso', 'data', 'numeros_sorteio', 'quantidade_ganhadores', 'premio'])

display(df)

In [ ]:
# insere os dados no DataFrame
def funcao_inserir_dados_no_dataframe():
    df.loc[concurso]=[concurso, data, numeros_sorteio, quantidade_ganhadores, premio]

funcao_inserir_dados_no_dataframe()
display(df)

In [ ]:
# cria variavel para contagem regressiva
proximo_concurso = concurso
print(proximo_concurso)

In [ ]:
# faz o looping até pegar todos os dados
while proximo_concurso > 1:
    proximo_concurso -= 1
    texto_proximo_concurso = str(proximo_concurso)
    
    try:
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').clear()
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').send_keys(proximo_concurso)
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').send_keys(Keys.ENTER)
        
        esperar = WebDriverWait(nav, 10).until(
            EC.text_to_be_present_in_element((By.XPATH, '//*[@id="conteudoresultado"]/div[1]/div/h2/span'), texto_proximo_concurso)
        )
        
    except:
        nav.refresh()
        
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').clear()
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').send_keys(proximo_concurso)
        nav.find_element_by_xpath('//*[@id="buscaConcurso"]').send_keys(Keys.ENTER)
        
        esperar = WebDriverWait(nav, 10).until(
            EC.text_to_be_present_in_element((By.XPATH, '//*[@id="conteudoresultado"]/div[1]/div/h2/span'), texto_proximo_concurso)
        )
    
    funcao_pegar_concurso_data()
    
    funcao_pegar_concurso()
    
    funcao_pegar_data()
    
    if concurso == proximo_concurso:

        funcao_pegar_numeros_sorteio()

        funcao_pegar_ganhador_premio()
        
        funcao_pegar_quantidade_ganhadores()
        
        funcao_pegar_premio()
        
        funcao_inserir_dados_no_dataframe()

    else:
        proximo_concurso += 1

print("Loop executado com sucesso!")

In [ ]:
# mostra o resultado final do looping com todos os resultados
display(df)

In [ ]:
# mostra algumas linhas aleatorias para verificar o resultado
df.sample(10)

In [ ]:
# separa os números sorteados
df = df.assign(n1 = df["numeros_sorteio"].str[0:2],
               n2 = df["numeros_sorteio"].str[2:4],
               n3 = df["numeros_sorteio"].str[4:6],
               n4 = df["numeros_sorteio"].str[6:8],
               n5 = df["numeros_sorteio"].str[8:10],
               n6 = df["numeros_sorteio"].str[10:12]
              )
display(df)

In [ ]:
# verifica as informações do DataFrame
df.info()

In [ ]:
# modifica o tipo dos dados no DataFrame
df[["n1", "n2", "n3", "n4", "n5", "n6"]] = df[["n1", "n2", "n3", "n4", "n5", "n6"]].astype('float').astype('Int8')
df.dtypes

In [ ]:
# cria um DataFrame para cada coluna dos números sorteados
df1 = pd.DataFrame(df['n1'])
df2 = pd.DataFrame(df['n2'])
df3 = pd.DataFrame(df['n3'])
df4 = pd.DataFrame(df['n4'])
df5 = pd.DataFrame(df['n5'])
df6 = pd.DataFrame(df['n6'])

In [ ]:
# renomeia as colunas dos números sorteados
df1 = df1.rename(columns={'n1': 'N'})
df2 = df2.rename(columns={'n2': 'N'})
df3 = df3.rename(columns={'n3': 'N'})
df4 = df4.rename(columns={'n4': 'N'})
df5 = df5.rename(columns={'n5': 'N'})
df6 = df6.rename(columns={'n6': 'N'})

In [ ]:
# junta todos os números sorteados em um único DataFrame
freq_df = pd.concat([df1, df2, df3, df4, df5, df6])
print(freq_df)

In [ ]:
# cria o gráfico de quantas vezes cada número já foi sorteado
freq_df.groupby(['N']).size().plot(kind = 'barh', figsize = (15, 15)).xaxis.grid(True)

In [ ]:
# salva o DataFrame em um arquivo de excel
df.to_excel("Todos_Os_Resultados_Da_Mega_Sena.xlsx", index=False)

In [ ]:
# fecha o navegador
nav.quit()

In [ ]:
# mostra a data e o horário final do código e quanto tempo ele demorou para rodar
fim_codigo = time.strftime('"%A, %d %B %Y, %H:%M:%S', time.localtime())

tempo_final = int(time.time())
diferenca_de_tempo = int(tempo_final - tempo_inicial)
tempo_total_de_execucao = time.strftime('%H:%M:%S', time.gmtime(diferenca_de_tempo))

print(inicio_codigo)
print(fim_codigo)
print(f'O código demorou no total: {tempo_total_de_execucao}')